In [1]:
# Fill in your name using the format below and student ID number
your_name = "CHU, VI"
student_id = "0890745"

In [2]:
# Before submission, set this to True so that you can render and verify this notebook without training deep learning models.
# Any deep learning models will be trained from file instead.
stop_training = True

In [3]:
# Uncomment the following line to run in Google Colab
# !pip install --quiet openml 

In [4]:
# Uncomment the following line to run in Google Colab
# %tensorflow_version 2.x
import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU') # Check whether GPUs are available

[]

In [5]:
%matplotlib inline
import openml as oml
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [6]:
from packaging import version
import sklearn
import tensorflow
sklearn_version = sklearn.__version__
tensorflow_version = tensorflow.__version__
if version.parse(sklearn_version) < version.parse("0.22.0"):
    print("scikit-learn is outdated. Please update now!")
if version.parse(tensorflow_version) < version.parse("2.1.0"):
    print("Tensorflow is outdated. This is version {}. Please update to 2.1.".format(tensorflow_version))
else:
    print("Hi{}, Looks good. You may continue :)".format(your_name.split(",")[1]))

Hi VI, Looks good. You may continue :)


# Assignment 3

### Choice of libraries
We recommend to use Tensorflow in this assignment since that is what we covered in the labs. If you feel confident using PyTorch (and Skorch for the scikit-learn wrapper), that is allowed too, as long as you are able to implement the requested functions and return the requested data. Read the assignment carefully and ensure that you can. Note that you may also need to do a bit more work to implement certain helper functions and wrappers.

### Storing and submitting files
You must be able to store your models and submit them to GitHub Classroom. The evaluation functions used in this notebook will automatically store models for you.

If you want to run and solve the notebook on your local machine/laptop, fill in the path 'base_dir' to your assignment folder into the next cell.

If you use Colab, we recommend that you link it to your Google Drive:  
* Upload the assignment folder to your Google Drive (+ New > Folder Upload)
* Open Colab in a browser, open the 'Files' menu in the left sidebar, and click 'Mount Drive'
  * At this point you may need to authenticate
* Fill in the path to your assignment folder below
  * It's likely '/content/drive/My Drive/assignment-3-yourname'

In [7]:
# base_dir = '/content/drive/My Drive/2imm15 Assignment 3'
 # For Google Colab
base_dir = 'C:/Users/s144195/OneDrive/1. TUe/2. Master/Y1Q3/Web Info & Data Mining (2IMM15)/4. Assignments/assignment-3-sushivi'

In [8]:
# Uncomment to link Colab notebook to Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

### Using GPUs
While you can solve this assignment on a CPU, using a GPU will speed things up training quite a bit. If you have a local GPU, you can use that. If you don't, we recommend Google Colab. When you are in Colab:
* In Runtime > Change runtime type, select the GPU under Hardware Accelerator
* Run the 3rd cell on the top of this notebook to check that the GPU is found.

Note that Colab may not always have GPUs ready all the time, and may deny you a GPU when you have used them a lot. When you are temporarily 'locked out', you can switch to a non-GPU runtime or to a local instance of Jupyter running on your machine.

### Constraints
* Your stored models should not be larger than 100MB when stored in file. GitHub will not allow uploading if they are.
* When questions ask you to provide an explanation, it should be less than 500
characters long. Some questions have a higher limit. Always answer in full sentences.
* Don't train for more than 100 epochs, i.e. don't throw excessing computational resources at the problem. If your model hasn't converged by then, think of ways it could be made to converge faster. In this assignment you are not after the last tiny improvement, you can stop when learning curves flatten out. Do at least 5 epochs to get a reasonable learning curve.

### Grading
Grading is based on the following aspects:
* Correctness in answering the question. Carefully read the question and answer
what is asked for. Train your models on the correct data. It should be clear on which data should be trained, but ask when in doubt. When something is not defined (e.g. the number of epochs or batch size), you can freely choose them.
* Clarity of your explanations. Write short but precise descriptions of what you did and why. Give short but clear explanations of the observed performance. 
After your explanation, your approach and model should make perfect sense. Refrain from using symbols as substitute for words in your explanation (e.g. no: "More layers -> more parameters" yes: "More layers mean more parameters"). 
* Part of your grade depends on how well your model performs. When the question says 'you should at least get x%', x% will give you a good but not the maximal grade. You can get the full grade when you are close to what is the expected maximal performance. You don't need to invest lots of effort into the last tiny improvement, though. Unless specified, we look at the accuracy on the validation set. If your learning curves are very erratic we'll compute a score based on the smoothed curves (i.e. single peaks don't count).
* The weight of each question is indicated. Take this into account when planning your time.

### Other tips
* Don't wait until the last minute to do the assignment. The models take time to train, most questions will require some thinking, and some require you to read up on some new concepts.
* Take care that you upload the results as requested. You need to submit not only the notebooks but also the trained models and learning curves (training histories). Be sure to run the verification script and check that all the results are included.
* We provide an evaluation function that also stored models to disk. After you are done training the model, set the 'train' attribute to False so that the model doesn't train again (and loads from file instead) when you restart and rerun your notebook.
* Explore. For many questions we'll ask you to explain your model design decisions. You cannot magically know the best solutions but you can experiment
based on your understanding and make decisions based on both your knowledge and experiments. Your explanation is at least as important as the performance of your model.
* Be original. We will check for plagiarism between student submissions.

### Data
The [Street View House Numbers Dataset](https://www.openml.org/d/41081) contains 32-by-32 RGB images centered around a single digit of a house number appearing in Google Street View. Many of the images do contain some distractors at the sides. It consists of 10 classes, 1 for each digit. Digit '1' has label 1, '9' has label 9 and '0' has label 10. Your goal is to build models that recognize the correct digit.

If you use Colab, uncomment the following to cache the dataset inside the VM. This will make reloading faster if you need to restart your notebook. After longer periods of inactivity, your VM may be recycled and the cache lost, in which case the dataset will be downloaded again. Also note that this dataset is about 1Gb large, and will take even more space in memory. You may need to switch to a high-RAM environment (Colab will ask you if you hit the limit).

In [9]:
# Use OpenML caching in Colab
# On your local machine, it will store data in a hidden folder '~/.openml'
import os
# oml.config.cache_directory = os.path.expanduser('/content/cache')

In [ ]:
# Download Streetview data. Takes a while (several minutes), and quite a bit of
# memory when it needs to download. After caching it loads faster.
SVHN = oml.datasets.get_dataset(41081)
X, y, _, _ = SVHN.get_data(dataset_format='array',
    target=SVHN.default_target_attribute)

Reshape, sample and split the data

In [ ]:
from tensorflow.keras.utils import to_categorical

Xr = X.reshape((len(X),32,32,3))
Xr = Xr / 255.
yr = to_categorical(y)

In [ ]:
# DO NOT EDIT. DO NOT OVERWRITE THESE VARIABLES.
from sklearn.model_selection import train_test_split
# We do an 80-20 split for the training and test set, and then again a 80-20 split into training and validation data
X_train_all, X_test, y_train_all, y_test = train_test_split(Xr,yr, stratify=yr, train_size=0.8, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train_all,y_train_all, stratify=y_train_all, train_size=0.8, random_state=1)
evaluation_split = X_train, X_val, y_train, y_val

Check the formatting - and what the data looks like

In [ ]:
from random import randint

# Takes a list of row ids, and plots the corresponding images
# Use grayscale=True for plotting grayscale images
def plot_images(X, y, grayscale=False):
    fig, axes = plt.subplots(1, len(X),  figsize=(10, 5))
    for n in range(len(X)):
        if grayscale:
            axes[n].imshow(X[n], cmap='gray')
        else:
            axes[n].imshow(X[n])
        axes[n].set_xlabel((np.argmax(y[n])+1)%10) # Label is index+1
        axes[n].set_xticks(()), axes[n].set_yticks(())
    plt.show();

images = [randint(0,len(X_train)) for i in range(5)]
X_random = [X_train[i] for i in images]
y_random = [y_train[i] for i in images]
plot_images(X_random, y_random)

### Evaluation harness
We provide an evaluation function 'run_evaluation' that you should use to 
evaluate all your models. It also stores the trained models to disk so that
your submission can be quickly verified, as well as to avoid having to train
them over and over again. Your last run of the evaluation function (the last one
stored to file), is the one that will be evaluated. The 'train' argument indicates whether to train or to load from disk. We have provided helper functions for saving and loading models to/from file, assuming you use TensorFlow. If you use PyTorch you'll have to adapt them.

In [ ]:
import os
import pickle
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model # for use with tensorflow

def shout(text, verbose=1):
    """ Prints text in red. Just for fun.
    """
    if verbose>0:
        print('\033[91m'+text+'\x1b[0m')

def load_model_from_file(base_dir, name, extension='.h5'):
    """ Loads a model from a file. The returned model must have a 'fit' and 'summary'
    function following the Keras API. Don't change if you use TensorFlow. Otherwise,
    adapt as needed. 
    Keyword arguments:
    base_dir -- Directory where the models are stored
    name -- Name of the model, e.g. 'question_1_1'
    extension -- the file extension
    """
    try:
        model = load_model(os.path.join(base_dir, name+extension))
    except OSError:
        shout("Saved model could not be found. Was it trained and stored correctly? Is the base_dir correct?")
        return False
    return model

def save_model_to_file(model, base_dir, name, extension='.h5'):
    """ Saves a model to file. Don't change if you use TensorFlow. Otherwise,
    adapt as needed. 
    Keyword arguments:
    model -- the model to be saved
    base_dir -- Directory where the models should be stored
    name -- Name of the model, e.g. 'question_1_1'
    extension -- the file extension
    """
    model.save(os.path.join(base_dir, name+extension))

# Helper function to extract min/max from the learning curves
def minMax(x):
    return pd.Series(index=['min','max'],data=[x.min(),x.max()])

# DO NOT EDIT
def run_evaluation(name, model_builder, data, base_dir, train=True, 
                   generator=False, epochs=3, batch_size=32, steps_per_epoch=60, 
                   verbose=1, **kwargs):
    """ Trains and evaluates the given model on the predefined train and test splits,
    stores the trained model and learning curves. Also prints out a summary of the 
    model and plots the learning curves.
    Keyword arguments:
    name -- the name of the model to be stored, e.g. 'question_1_1.h5'
    model_builder -- function that returns an (untrained) model. The model must 
                     have a 'fit' function that follows the Keras API. It can wrap
                     a non-Keras model as long as the 'fit' function takes the 
                     same attributes and returns the learning curves (history).
                     It also must have a 'summary' function that prints out a 
                     model summary, and a 'save' function that saves the model 
                     to disk. 
    data -- data split for evaluation. A tuple of either:
            * Numpy arrays (X_train, X_val, y_train, y_val)
            * A data generator and validation data (generator, X_val, y_val)
    base_dir -- the directory to save or read models to/from
    train -- whether or not the data should be trained. If False, the trained model
             will be loaded from disk.
    generator -- whether the data in given as a generator or not
    epochs -- the number of epochs to train for
    batch_size -- the batch size to train with
    steps_per_epoch -- steps per epoch, in case a generator is used (ignored otherwise)
    verbose -- verbosity level, 0: silent, 1: minimal,...
    kwargs -- keyword arguments that should be passed to model_builder.
              Not required, but may help you to adjust its behavior
    """
    model = model_builder(**kwargs)
    if not model:
        shout("No model is returned by the model_builder")
        return
    if not hasattr(model, 'fit'):
        shout("Model is not built correctly")
        return
    learning_curves = {}
    if train and not stop_training: # Train anew
        shout("Training the model", verbose)
        if generator:
            generator, X_val, y_val = data
            history = model.fit(generator, epochs=epochs, batch_size=batch_size,
                              steps_per_epoch=steps_per_epoch, verbose=1, 
                              validation_data=(X_val, y_val))
            learning_curves = history.history
        else:
            X_train, X_val, y_train, y_val = data
            history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                              verbose=1, validation_data=(X_val, y_val))
            learning_curves = history.history
        shout("Saving to file", verbose)
        save_model_to_file(model, base_dir, name)
        with open(os.path.join(base_dir, name+'.p'), 'wb') as file_pi:
            pickle.dump(learning_curves, file_pi)
        shout("Model stored in "+base_dir, verbose)
    else: # Load from file
        shout("Loading model from file", verbose)
        model = load_model_from_file(base_dir, name)
        if not model:
            shout("Model not found")
        return
        learning_curves = None
        try:
            learning_curves = pickle.load(open(os.path.join(base_dir, name+'.p'), "rb"))
        except FileNotFoundError:
            shout("Learning curves not found")
        return
        shout("Success!", verbose)
    # Report
    print(model.summary())
    lc = pd.DataFrame(learning_curves)
    lc.plot(lw=2,style=['b:','r:','b-','r-']);
    plt.xlabel('epochs');
    print(lc.apply(minMax))

In [ ]:
# Toy usage example
# Remove before submission
from tensorflow.keras import models
from tensorflow.keras import layers 

def build_toy_model():
    model = models.Sequential()
    model.add(layers.Reshape((3072,), input_shape=(32,32,3)))
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# First build and store
run_evaluation("toy_example", build_toy_model, evaluation_split, base_dir, 
               train=False, epochs=3, batch_size=32)

In [ ]:
# Toy usage example
# Remove before submission
# With train=False: load from file and report the same results without rerunning
run_evaluation("toy_example", build_toy_model, evaluation_split, base_dir, 
               train=False)

## Part 1. Dense networks (10 points)

### Question 1.1: Baseline model (4 points)
- Build a dense network (with only dense layers) of at least 3 layers that is shaped like a pyramid: The first layer must have many nodes, and every subsequent layer must have increasingly fewer nodes, e.g. half as many. Implement a function 'build_model_1_1' that returns this model.
- You can explore different settings, but don't use any preprocessing or regularization yet. You should be able to achieve at least 70% accuracy, but more is of course better. Unless otherwise stated, you can use accuracy as the evaluation metric in all questions.
* Add a small description of your design choices (max. 500 characters) in 'answer_q_1_1': explain what you did and also why. Also discuss the performance of the model. Is it working well? Both the performance of the model and your explanations matter.

- The name of the model should be 'model_1_1'. Evaluate it using the 'run_evaluation' function. For this question, you should not use more than 50 epochs.

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers 

def build_model_1_1():
    model = models.Sequential()
    model.add(layers.Reshape((3072,), input_shape=(32,32,3)))
    model.add(layers.Dense(1000, activation='sigmoid'))
    model.add(layers.Dense(500, activation='sigmoid'))
    model.add(layers.Dense(250, activation='sigmoid'))
    model.add(layers.Dense(100, activation='sigmoid'))
    model.add(layers.Dense(10, activation='sigmoid'))
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

run_evaluation("model_1_1", build_model_1_1, evaluation_split, base_dir, 
               train=True, steps_per_epoch = 30, epochs=20, batch_size=50)
answer_q_1_1 = """
               Since our input does not contain negative values, I have chosen
               to not use any tanh activation. So all layers consist of a sigmoid
               or relu activation. Trying layers in pyramid shape consisting of all relu values 
               fluctuated around the same accuracy 0.4196 as maximum within 40 epochs. This method was not a succes, so we try something different
               Trying only sigmoid activation: a quick improvement on accuracy within 10 epochs and then starts stagnating after 20 epochs, 
               achieved  val_accuracy of 77.22% in 10th epoch.
               Trying combination of layers: performed not quite well.
               """
print("Answer is {} characters long".format(len(answer_q_1_1)))

### Question 1.2: Preprocessing (2 points)
Rerun the model, but now preprocess the data first by converting the images to 
greyscale. You can use the helper function below. If you want to do additional 
preprocessing, you can do that here, too.
* Store the preprocessed data as a tuple `preprocessed_split`
* Rerun and re-evaluate your model using the preprocessed data.
  * For the remainder of the assignment, always use the preprocessed data
* Explain what you did and interpret the results in 'answer_q_1_2'. Is the model
  better, if so, why?

In [ ]:
# Luminance-preserving RGB to greyscale conversion
def rgb2gray(X):
  return np.expand_dims(np.dot(X, [0.2990, 0.5870, 0.1140]), axis=3)

In [ ]:
# Replace with the preprocessed data
preprocessed_split = rgb2gray(X_train), rgb2gray(X_val), y_train, y_val

# Adjusted model
def build_model_1_2():
    model = models.Sequential()
    model.add(layers.Reshape((32*32,), input_shape=(32,32,1)))
    model.add(layers.Dense(1000, activation='sigmoid'))
    model.add(layers.Dense(500, activation='sigmoid'))
    model.add(layers.Dense(250, activation='sigmoid'))
    model.add(layers.Dense(100, activation='sigmoid'))
    model.add(layers.Dense(10, activation='sigmoid'))
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Evaluate. Use a new name 'model_1_2' to not overwrite the previous trained model
run_evaluation("model_1_2", build_model_1_2, preprocessed_split, base_dir, 
               train=True, steps_per_epoch = 20, epochs=20, batch_size=50)
answer_q_1_2 = """
                RGB to greyscale conversion resulted in somewhat the same performance of the model, 
                but it reduce the computing time significantly and the model started converging a bit faster. 
                Other modifications to the model, like adding a dense layer with relu activation worsened the performance of the model. 
               """
print("Answer is {} characters long".format(len(answer_q_1_2)))

### Question 1.3: Regularization and tuning (4 points)
* Regularize the model. You can explore (and combine) different techniques. What works best?
* Tune other hyperparameters (e.g. learning rate, batch size,...) as you see fit.
* Explain your findings and final design decisions. Retrain the model again on the preprocessed data and discuss the results.
* Return your model in function 'build_model_1_3' and write your answer in 'answer_q_1_3'

In [ ]:
from tensorflow.keras import regularizers

def build_model_1_3():
    model = models.Sequential()
    model.add(layers.Reshape((32*32,), input_shape=(32,32,1)))
    model.add(layers.Dense(1500, activation='sigmoid', kernel_regularizer=regularizers.l2(0.000001)))
    model.add(layers.Dropout(0.2))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(1000, activation='sigmoid', kernel_regularizer=regularizers.l2(0.000001)))
    model.add(layers.Dropout(0.2))
    # model.add(layers.BatchNormalization())
    model.add(layers.Dense(500, activation='sigmoid', kernel_regularizer=regularizers.l2(0.000001)))
  
    model.add(layers.Dense(250, activation='sigmoid', kernel_regularizer=regularizers.l2(0.000001)))
    model.add(layers.Dense(100, activation='sigmoid', kernel_regularizer=regularizers.l2(0.000001)))
    # model.add(layers.Dense(50, activation='sigmoid' ))
    model.add(layers.Dense(10, activation='sigmoid', kernel_regularizer=regularizers.l2(0.000001)))
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

run_evaluation("model_1_3", build_model_1_3, preprocessed_split, base_dir, 
               train=True, steps_per_epoch = 200, epochs=25, batch_size=32)

answer_q_1_3 = """
               First we play with the number of nodes, to find a sweet spot. 
               We start with 500 nodes instead of 1000: This works very well and performs even better reaching val_acc of 81.19%
               Making a smaller step between 100 to 10 nodes: This does not necesarilly perform better, so we discard the layeragain. 
               Now we try weight regularization: l2 with parameter 0.001 performs very badly. Changing the parameter to 0.000001 works better! 82.17%. l1 regularization or the combination between l1 and l2 did not perform better.
               Next trying dropout with parameter 0.5, which performs worse. Dropout with parameter 0.2 performs better.
               Batch normalization also performs better. 
               Trying to make the model more complex with the regularization does not work very well, so we keep the layers as it is. 
               
               """
print("Answer is {} characters long".format(len(answer_q_1_3)))

## Part 2. Convolutional neural networks (10 points)
### Question 2.1: Design a ConvNet (7 points)
- Build a sequential convolutional neural network. Try to achieve the best validation accuracy you can. You should be able to get at least 90% accuracy. You can use any depth, any combination of layers, and any kind of regularization and tuning. 
- Add a description of your design choices in 'answer_q_2_1': explain what you did and also why. Also discuss the performance of the model. Is it working well? Both the performance of the model and your explanations matter.
- You are allowed **800** characters for this answer (but don’t ramble).
- The name of the model should be 'model_2_1'. Evaluate it using the 'run_evaluation' function and the preprocessed data.

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import regularizers

# def build_model_2_1():
#   model = models.Sequential()
#   # model.add(layers.Reshape((32*32,), input_shape=(32,32,1)))
#   model.add(layers.Conv2D(32, (3), activation='relu', input_shape=(32,32,1)))
#   model.add(layers.Dropout(0.2))
#   model.add(layers.MaxPooling2D((2, 2)))
#   model.add(layers.Conv2D(64, (3, 3), activation='relu'))
#   model.add(layers.MaxPooling2D((2, 2)))
#   model.add(layers.Dropout(0.2))
#   model.add(layers.Conv2D(128, (3, 3), activation='relu'))
#   model.add(layers.MaxPooling2D((2, 2)))
#   model.add(layers.Dropout(0.2))
#   model.add(layers.Flatten())
#   model.add(layers.Dense(128, activation='sigmoid', kernel_regularizer=regularizers.l2(0.0001)))
#   # model.add(layers.Dropout(0.2))
#   model.add(layers.Dense(50, activation='sigmoid', kernel_regularizer=regularizers.l2(0.0001)))
#   model.add(layers.Dense(10, activation='sigmoid', kernel_regularizer=regularizers.l2(0.0001)))
#   # model.add(layers.Dropout(0.2))
#   # model.add(layers.Dense(10, activation='sigmoid', kernel_regularizer=regularizers.l2(0.0001)))
#   model.compile(optimizer='rmsprop',
#                 loss='categorical_crossentropy',
#                 metrics=['accuracy'])
#   return model

def build_model_2_1():
    model = models.Sequential()
    # model.add(layers.Reshape((32*32,), input_shape=(32,32,1)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 1)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
    model.add(layers.Dropout(0.1))
    # model.add(layers.Dense(50, activation='sigmoid', kernel_regularizer=regularizers.l2(0.0001)))
    model.add(layers.Dense(10, activation='softmax', kernel_regularizer=regularizers.l2(0.0001)))
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

run_evaluation("model_2_1", build_model_2_1, preprocessed_split, base_dir, 
               train=True,steps_per_epoch = 400, epochs=10, batch_size=32)
answer_q_2_1 = """
               Trying 2Dconvolution with relu: 89,93% it started overfitting after around 10 epochs. 
               2D convolution with sigmoid: 91,55%, starts overfitting a bit.
               Adding dropout and regularizer to go against overfitting: 92,38% but less overfitting!
               Trying VGC like model with deeper layers: 93,61%. This is the best performing model yet, so we decide to use this model. 

               """
print("Answer is {} characters long".format(len(answer_q_2_1)))

### Question 2.2: Data Augmentation (3 points)

- Augment the preprocessed training data. You can explore using image shifts, rotations, zooming, flips, etc. What works well, and what does not?
- Evaluate the model from question 2.1 with the augmented data using the 'run_evaluation' function. Store the new trained model as 'model_2_2'.
- Add a description of your design choices in 'answer_q_2_2': explain what you did and also why. Also discuss the performance of the model.

In [ ]:
# Do some date augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_dir = '/content/drive/My Drive/2imm15 Assignment 3/augmentation/train'
validation_dir = '/content/drive/My Drive/2imm15 Assignment 3/augmentation/validation'

train_datagen = ImageDataGenerator(
    rotation_range=[-5,5],
    width_shift_range=5,
    height_shift_range=5,
    shear_range=0.5, 
    zoom_range=0.5, 
    brightness_range=[-5.0, 5.0],
    horizontal_flip=True,
    vertical_flip = True,
    fill_mode='reflect')

data_gen = ImageDataGenerator()

data_generator = data_gen.flow(X_train, y_train, batch_size=32)
data_listX = []
data_listy = []
batch_index = 0

while batch_index <= data_generator.batch_index:
  dataX, datay = data_generator.next()
  data_listX.append(dataX[0])
  data_listy.append(datay[0])
  batch_index += 1

data_arrayX = np.asarray(data_listX)
data_arrayy = np.asarray(data_listy)

augmented_train = train_datagen.flow(rgb2gray(data_arrayX), data_listy, batch_size=32)


In [ ]:
# Note that we build the same untrained model as in question 2.1 but store the 
# trained version as model_2_2. Change attributes as needed to run on augmented
# data

augmented_split = augmented_train.x , rgb2gray(X_val), augmented_train.y, y_val
run_evaluation("model_2_1", build_model_2_1, augmented_split, base_dir, 
               train=True,steps_per_epoch = 600, epochs=30, batch_size=32)
answer_q_2_2 = """
               Tried an initial setting first for: rotation, width, height, shear, zoom, brightness and fill. This did not perform better than without augmentation. 
               Now we're trying eacht augmentation separately. Rotation augmentation performs worse. The val_accuracy is fluctuating a lot. But best accuracy lays around 75%.
               From pictures I've noticed that widht, height, rotation, shear and zoom can vary quite a lot in digits. Trying to play with this number, slowly leads to a better result. 
                But it seems like the model is overfitting now. 
               """
print("Answer is {} characters long".format(len(answer_q_2_2)))

## Part 3. Model interpretation (10 points)
### Question 3.1: Interpreting misclassifications (2 points)
Study which errors are still made by your last model (model_2_2) by evaluating it on the test data. You do not need to retrain the model.
* What is the accuracy of model_2_2 on the test data? Store this in 'test_accuracy_3_1'.
* Plot the confusion matrix in 'plot_confusion_matrix' and discuss which classes are often confused.
* Visualize the misclassifications in more depth by focusing on a single
class (e.g. the number '2') and analyse which kinds of mistakes are made for that class. For instance, are the errors related to the background, noisiness, etc.? Implement the visualization in 'plot_misclassifications'.
* Summarize your findings in 'answer_q_3_1'

In [ ]:
model = load_model_from_file(base_dir, "model_2_2", extension='.h5')
y_pred = model.predict(rgb2gray(X_test))
misclassified_samples = np.nonzero(np.argmax(y_test, axis=1) != np.argmax(y_pred, axis=1))[0]
misclassified_samples_5 = []
# print(y_test[4][0])
for i in misclassified_samples: 
  if y_test[i][4] == 1.0:
    misclassified_samples_5.append(i)


In [ ]:
augmented_split = augmented_train.x , rgb2gray(X_test), augmented_train.y, y_test
run_evaluation("model_2_1", build_model_2_1, augmented_split, base_dir, 
               train=True,steps_per_epoch = 400, epochs=15, batch_size=32)

In [ ]:
from sklearn.metrics import confusion_matrix
import random

test_accuracy_3_1 = 1- len(misclassified_samples)/len(y_test)
print(test_accuracy_3_1)
def plot_confusion_matrix(y_test, y_pred):
  cm = confusion_matrix(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1))
  fig, ax = plt.subplots()
  im = ax.imshow(cm)
  ax.set_xticks(np.arange(10)), ax.set_yticks(np.arange(10))
  ax.set_xticklabels([1,2,3,4,5,6,7,8,9,10], rotation=45, ha="right")
  ax.set_yticklabels([1,2,3,4,5,6,7,8,9,10])
  ax.set_ylabel('True')
  ax.set_xlabel('Predicted')
  for i in range(100):
      ax.text(int(i/10),i%10,cm[i%10,int(i/10)], ha="center", va="center", color="w")

plot_confusion_matrix(y_test,y_pred)


def plot_misclassifications():
  fig, axes = plt.subplots(1, 5,  figsize=(10, 5))
  random_misclassified_samples = random.sample(misclassified_samples_5, 5)
  for nr, i in enumerate(random_misclassified_samples):
      axes[nr].imshow(X_test[i])
      axes[nr].set_xlabel("Predicted: %s,\n Actual : %s" % ((np.argmax(y_pred[i]) + 1)%10, (np.argmax(y_test[i]) + 1)%10))
      axes[nr].set_xticks(()), axes[nr].set_yticks(())

  plt.show()

# Visualize the randomly five misclassifications of the digit 5, together with the predicted and actual class
plot_misclassifications() 

answer_q_3_1 = """
                The test accuracy is 0.913. 
               The digit 5 got confused with a 3 the most. Because of the shape, 
               it is understandable that these digits get confused. Looking at random plots of misclassifications, it looks like most of the mistakes are due to noisiness. 
               However, a the prediction 3 sometimes happen even if it is not noisy at all.
               """
print("Answer is {} characters long".format(len(answer_q_3_1)))

### Question 3.2: Visualizing activations (4 points)
* Implement a function `plot_activations()` that returns the most interesting activations (feature maps). Select the first example from the test set. Retrieve and visualize the activations of model 2_2 for that example (make sure you load that model in the function), for every filter for different convolutional layers (at different depths in the network).
* Give an explanation (as detailed as you can) about your observations in 'answer_q_3_2'. Is your model indeed learning something useful?

In [ ]:
# model.summary()

In [ ]:
# We preprocess the image into a 4D tensor
from tensorflow.keras.preprocessing import image
img_tensor = rgb2gray(X_test)[0]
img_tensor = np.expand_dims(img_tensor, axis=0) 
img_tensor /= 255.
# Extracts the outputs of the top 8 layers:
layer_outputs = [layer.output for layer in model.layers[:15]]
# Creates a model that will return these outputs, given the model input:
activation_model = models.Model(inputs=model.input, outputs=layer_outputs)

# This will return a list of 5 Numpy arrays:
# one array per layer activation
activations = activation_model.predict(img_tensor)

In [ ]:
images_per_row = 10
def plot_activations(layer_index, activations):
    start = layer_index
    end = layer_index+1
    # Now let's display our feature maps
    for layer_activation in activations[start:end]:
      # This is the number of features in the feature map
      n_features = layer_activation.shape[-1]

      # The feature map has shape (1, size, size, n_features)
      size = layer_activation.shape[1]

      # We will tile the activation channels in this matrix
      n_cols = n_features // images_per_row
      display_grid = np.zeros((size * n_cols, images_per_row * size))

      # We'll tile each filter into this big horizontal grid
      for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = layer_activation[0,
                                                 :, :,
                                                 col * images_per_row + row]
            # Post-process the feature to make it visually palatable
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size,
                             row * size : (row + 1) * size] = channel_image

      # Display the grid
      scale = 1. / size
      plt.figure(figsize=(scale * display_grid.shape[1],
                            scale * display_grid.shape[0]))
      plt.title("Activation of layer {}".format(layer_index+1))
      plt.grid(False)
      plt.imshow(display_grid, aspect='auto', cmap='viridis')

    plt.show()
for i in range(len(activations)):
  plot_activations(i, activations)

answer_q_3_2 = """
               It seems like the first layer is doing some edge detection which is useful. 
               Then the 4 layers after that are some more abstract versions of edge detection.
               Layers 6 up to 8, mainly highlighting the parts of the edge of the image, which doesn't seem very useful. 
               Layer 9 to 12 are very abstract higlights of different parts of the image. This could be detecting different parts of the digit. 
               """
print("Answer is {} characters long".format(len(answer_q_3_2)))

### Question 3.3: Visualizing activations (4 points)
* Again, select the first example from the test set, and the trained model_2_2.
* Implement a function `plot_activation_map()` that builds and shows a class activation map for your last convolutional layer that highlights what the model is paying attention to when classifying the example.
* If possible, superimpossible the activation map over the image. If not, plot
them side by side. Implement a function 'plot_3_3' that returns the entire plot.

In [ ]:
def plot_activation_map(): 
  
  pass

def plot_3_3():

  pass

## Part 4. Transfer learning (10 points)
### Question 4.1 Fast feature extraction with VGG16 (5 points)
- Import the VGG16 model, pretrained on ImageNet. [See here](https://keras.io/applications/). Only import the convolutional part, not the dense layers.
- Implement a function 'build_model_4_1` that adds a dense layer to the convolutional base, and freezes the convolutional base. Consider unfreezing the last few convolutional layers and evaluate whether that works better.
- Train the resulting model on the *original* (colored) training data
- Evaluate the resulting model using 'run_evaluate'. Discuss the observed performance in 'answer_q_4_1'.

In [ ]:
import tensorflow.keras as keras
conv_base = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=(32, 32, 3), pooling=None, classes=1000)

In [ ]:
conv_base.summary()

In [ ]:
def build_model_4_1():
  model = models.Sequential()
  model.add(conv_base)
  model.add(layers.Flatten())
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
  conv_base.trainable = False
  return model

run_evaluation("model_4_1", build_model_4_1, augmented_split, base_dir, 
               train=True,steps_per_epoch = 400, epochs=15, batch_size=32)
answer_q_4_1 = """
               Your answer 
               """
print("Answer is {} characters long".format(len(answer_q_4_1)))

### Question 4.2 Embeddings and pipelines (5 points)
- Generate embeddings of the original images by running them through the trained convolutional part of model_4_1 (without the dense layer) and returning the output. Embed the training and test data and store them to disk using the helper functions below. Implement a function `store_embeddings` that loads model_4_1 and stores all necessary embeddings to file. Make sure to run it once so that the embeddings are stored (and submitted).
- Implement a function 'generate_pipeline' that returns an scikit-learn pipeline. You can use any non-deep learning technique (eg. SVMs, RFs,...), and preprocessing technique. You can do model selection using the validation set. 
- Implement a function 'evaluate_pipeline' that evaluates a given pipeline on a given training and test set. 
- Implement a function 'evaluation_4_2' that evaluates your pipeline on the embedded training and test set (loaded from file) and returns the accuracy. 
- Describe what you did and what you observed. Report the obtained accuracy score. Can you beat your best model thus far?

In [ ]:
import pickle
import gzip

def store_embedding(X, name):  
  with gzip.open(os.path.join(base_dir, name+'_embedding.p'), 'wb') as file_pi:
    pickle.dump(X, file_pi)

def load_embedding(name):
  with gzip.open(os.path.join(base_dir, name+'_embedding.p'), 'rb') as file_pi:
    return pickle.load(file_pi)

def store_embeddings():
  """ Stores all necessary embeddings to file
  """
  pass

def generate_pipeline():
  """ Returns an sklearn pipeline.
  """
  pass

def evaluate_pipeline(pipeline, X_train, y_train, X_test, y_test):
  """ Evaluates the given pipeline, trained on the given embedded training set and 
      evaluated on the supplied embedded test set. Returns the accuracy score.
  """
  pass

def evaluation_4_2(X_train, y_train, X_test, y_test):
  """ Runs 'evaluate_pipeline' with embedded versions of the input data 
  and returns the accuracy.
  """
  pass

answer_q_4_2 = """
               Your answer 
               """
print("Pipeline:",generate_pipeline())
print("Answer is {} characters long".format(len(answer_q_4_2)))

In [ ]:
import os
import nbformat
from nbconvert import PythonExporter, HTMLExporter
from nbconvert.preprocessors import CellExecutionError, ExecutePreprocessor
from traitlets.config import Config
import warnings
warnings.filterwarnings('ignore') 

# Convert notebook to python
assignment_path = 'Assignment 3.ipynb'
export_path = 'solution.py'
template_path = './submit/Template.ipynb'
submission_path = 'Submission.html'

print("Converting: {0}".format(assignment_path))
exporter = PythonExporter()
# source is a tuple of python source code, meta contains metadata
(source, meta) = exporter.from_filename(assignment_path)
with open(export_path, 'w+') as fh:
    fh.writelines(source)
    fh.writelines("last_edit = '{}'".format(meta['metadata']['modified_date']))
    
# Run solution notebook
with open(template_path) as f:
    snb = nbformat.read(f, as_version=4)
ep = ExecutePreprocessor(timeout=2000, kernel_name='python3')

try:
    print("Running notebook... (may take a while)")
    out = ep.preprocess(snb, {'metadata': {'path': './'}})
except CellExecutionError:
    out = None
    msg = 'Error executing the notebook "%s".\n\n' % template_path
    msg += 'See notebook "%s" for the traceback.' % template_path
    print(msg)
    raise
finally:
    # Save notebook
    with open(template_path, mode='w', encoding='utf-8') as f:
        nbformat.write(snb, f)

# Export as HTML (PDF is too much hassle)
print("All good. Building report.")
c = Config()
c.TagRemovePreprocessor.enabled=True
c.TagRemovePreprocessor.remove_input_tags = set(["hide_input"])
c.preprocessors = ["TagRemovePreprocessor"]

html_exporter = HTMLExporter(config=c)
html_data, resources = html_exporter.from_notebook_node(snb)
html_data = html_data.replace('</head>', '<style>pre{font-family: "Times New Roman", Times, serif;}</style></head>')

with open(submission_path, "wb") as f:
    f.write(html_data.encode('utf8'))
    f.close()
#Cleanup
#os.remove("solution.py")
print("Done.")